In [1]:
!pip install gdown -U
!pip install evaluate

In [2]:
import os
os.environ["WANDB_DISABLED"]="true"

In [3]:
!mkdir -p results_temas_bert

In [4]:
!gdown '1pl9hGX3_4lMJvl4thbwkEXRCTi_dH1au'

Downloading...
From: https://drive.google.com/uc?id=1EjMX7Y6H5adIE2VqgnYuD6kzNOkmeUu3
To: /kaggle/working/my_Dataset_29JUL.csv
100%|█████████████████████████████████████████| 437k/437k [00:00<00:00, 108MB/s]


Aqui empiezo el código igual al de colab

In [5]:
from datasets import load_dataset

In [6]:
FILE_REQ='/kaggle/working/ReWoRC_Dataset_req.csv'
data_req = load_dataset('csv', data_files=FILE_REQ, sep=';')
print(data_req['train']['NFR'][21])

def create_labels(row):
  return {'id_doc': (
            'INT' if 'INT' in row['Referencia'] else
            'JUE' if 'JUE' in row['Referencia'] else
            'MON' if 'MON' in row['Referencia'] else
            'CON' if 'CON' in row['Referencia'] else
            'GES' if 'GES' in row['Referencia'] else
            'OTR' if 'OTR' in row['Referencia'] else
            'error'
        )}
data_req=data_req.map(lambda r: {'labels': r['NFR']})

data_req = data_req.map(create_labels)
data_req = data_req.class_encode_column('labels')
data_req = data_req.class_encode_column('id_doc')
data_req = data_req['train']
data_req.features

No funcional


{'Num': Value(dtype='int64', id=None),
 'Referencia': Value(dtype='string', id=None),
 'Requisito': Value(dtype='string', id=None),
 'Documento': Value(dtype='string', id=None),
 'NFR': Value(dtype='string', id=None),
 'labels': ClassLabel(names=['Funcional', 'No funcional'], id=None),
 'id_doc': ClassLabel(names=['CON', 'GES', 'INT', 'JUE', 'MON', 'OTR'], id=None)}

In [7]:
from transformers import AutoTokenizer
#BERT: model_name = 'google-bert/bert-base-cased'
#DEBERTA: model_name = 'microsoft/deberta-v3-base'
#ROBERTA: model_name = 'FacebookAI/roberta-base'
model_name = 'google-bert/bert-base-cased' # modificar con el modelo que queramos cada vez

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
def encode_text(row, tokenizer):
  row_encode = tokenizer(row['Requisito'], truncation=True)
  return row_encode


encode_data = data_req.map(encode_text, fn_kwargs={'tokenizer':tokenizer})
encode_data

from sklearn.model_selection import KFold
import numpy as np

folds = KFold(n_splits=6, shuffle=True)

splits = folds.split(np.zeros(6))
test_sets = []
train_val_sets = []
i=1
for train_val_idxs, test_idxs in splits:
  print (i)
  print(test_idxs)
  print('*******')
  data_test = encode_data.filter(lambda r : r['id_doc'] in test_idxs)
  data_train = encode_data.filter(lambda r : r['id_doc'] in train_val_idxs)
  data_train_val = data_train.train_test_split(train_size=0.9, stratify_by_column='labels')
  print(data_test)
  print(data_train)
  print(data_test.num_rows + data_train.num_rows)
  print(encode_data.num_rows == (data_test.num_rows + data_train.num_rows))
  print("***")


  test_sets.append(data_test)
  train_val_sets.append(data_train_val)
  i = i + 1

1
[5]
*******
Dataset({
    features: ['Num', 'Referencia', 'Requisito', 'Documento', 'NFR', 'labels', 'id_doc', 'input_ids', 'attention_mask'],
    num_rows: 893
})
Dataset({
    features: ['Num', 'Referencia', 'Requisito', 'Documento', 'NFR', 'labels', 'id_doc', 'input_ids', 'attention_mask'],
    num_rows: 1498
})
2391
True
***
2
[2]
*******
Dataset({
    features: ['Num', 'Referencia', 'Requisito', 'Documento', 'NFR', 'labels', 'id_doc', 'input_ids', 'attention_mask'],
    num_rows: 173
})
Dataset({
    features: ['Num', 'Referencia', 'Requisito', 'Documento', 'NFR', 'labels', 'id_doc', 'input_ids', 'attention_mask'],
    num_rows: 2218
})
2391
True
***
3
[0]
*******
Dataset({
    features: ['Num', 'Referencia', 'Requisito', 'Documento', 'NFR', 'labels', 'id_doc', 'input_ids', 'attention_mask'],
    num_rows: 110
})
Dataset({
    features: ['Num', 'Referencia', 'Requisito', 'Documento', 'NFR', 'labels', 'id_doc', 'input_ids', 'attention_mask'],
    num_rows: 2281
})
2391
True
***
4

In [10]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
import evaluate
import numpy as np

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return metric.compute(predictions=predictions, references=labels)

# load metric
metric_name = 'f1'
metric = evaluate.load(metric_name)

In [13]:
import glob
from transformers import TrainingArguments, Trainer
from sklearn.metrics import classification_report

lr_values =[2e-5, 5e-5]

for j in range(len(train_val_sets)):
    best_lr = -1
    best_f1 = -1

    # para escribir los resultados
    with open(f'results_temas_bert/res_temas_bert_{j}.txt', 'w') as f:
    # NOTA: si no va, deshacer el tab extra del siguiente bucle for
      for i, lr in enumerate(lr_values):
        print("================================================")
        print("j = " + str(j) + " // lr = " + str(lr))
        print("================================================")
      # argumentos para el entrenamiento
        training_args = TrainingArguments(
            #output_dir="my_checkpoint"+str(j),
            output_dir=f"my_checkpoint_{i}",
            overwrite_output_dir = True,
            num_train_epochs=10,
            #para probar tanto promise como nuestro coger 10 epochs
            evaluation_strategy="epoch",
            save_strategy="epoch",
            save_total_limit = 1,
            load_best_model_at_end=True,
            logging_strategy='epoch',
            optim="adamw_torch",
            per_device_train_batch_size=32,
            #lo dejaremos en 32
            #per_device_eval_batch_size=32,
            learning_rate=lr,
            #probar tanto 2e-5 como 5e-5
            weight_decay=0.01,
        )

        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
        # declarar el "entrenador"
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_val_sets[j]['train'],
            eval_dataset=train_val_sets[j]['test'],
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        # realizar el entrenamiento
        trainer.train()

        out_pred = trainer.predict(data_train_val['test'])
        if out_pred.metrics['test_f1'] > best_f1:
          best_f1 = out_pred.metrics['test_f1']
          best_lr = i
      # cargar el mejor modelo y resultados para test
      best_dir = glob.glob(f"my_checkpoint_{best_lr}/checkpoint*")
      print(best_dir)
      model_best = AutoModelForSequenceClassification.from_pretrained(best_dir[0])
      trainer_best = Trainer(
            model=model_best,
            args=None,
            train_dataset=None,
            eval_dataset=None,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

      test_data = test_sets[j]
      outputs_pred = trainer_best.predict(test_dataset=test_data)

      # model predictions
      predictions = np.argmax(outputs_pred.predictions, axis=1)

      #d_res = classification_report(test_data['labels'], predictions, digits=3, return_dict = True)
      d_res = classification_report(test_data['labels'], predictions, digits=3, output_dict=True)
      print(d_res)
      f.write("================== Best lr ================\n")
      f.write(str(best_lr))
      f.write("\n")
      f.write( "================== Resultados ================\n")
      f.write(str(d_res))

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


j = 0 // lr = 2e-05


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.376300,0.203839,0.645161
2,0.147700,0.135054,0.810811
3,0.080300,0.194660,0.764706
4,0.053900,0.222367,0.769231
5,0.037400,0.205316,0.810811
6,0.009800,0.288437,0.750000
7,0.006000,0.275169,0.787879
8,0.008200,0.292314,0.787879
9,0.002800,0.274024,0.787879
10,0.000400,0.280286,0.787879


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

j = 5 // lr = 5e-05


Epoch,Training Loss,Validation Loss,F1
1,0.374700,0.272207,0.633333
2,0.233300,0.190932,0.776119
3,0.148000,0.239518,0.754098
4,0.130100,0.564406,0.601942
5,0.070600,0.240036,0.811594
6,0.045300,0.239640,0.833333
7,0.019100,0.241186,0.873239
8,0.007600,0.304903,0.837838
9,0.008000,0.277259,0.873239
10,0.003400,0.281986,0.873239


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


['my_checkpoint_0/checkpoint-118']


{'0': {'precision': 0.9849056603773585, 'recall': 0.9125874125874126, 'f1-score': 0.9473684210526316, 'support': 286}, '1': {'precision': 0.5370370370370371, 'recall': 0.8787878787878788, 'f1-score': 0.6666666666666667, 'support': 33}, 'accuracy': 0.9090909090909091, 'macro avg': {'precision': 0.7609713487071978, 'recall': 0.8956876456876457, 'f1-score': 0.8070175438596492, 'support': 319}, 'weighted avg': {'precision': 0.9385744234800839, 'recall': 0.9090909090909091, 'f1-score': 0.9183303085299457, 'support': 319}}


Aquí acaba el código de colab. Falta hacer el zip para poder descargar los resultados (y opcionalmente usar el bot para que me avise del final de la ejecución)

In [14]:
import requests

# Definir la URL base y los parámetros
url = "https://api.callmebot.com/whatsapp.php"
params = {
    "phone": "+34653508040",
    "apikey": "1512070",
    "text": "🤖¡Ejecución terminada! 🤖\nVe a echarle un vistazo ☝🤓"
}

# Enviar la solicitud GET
response = requests.get(url, params=params)

# Imprimir el estado de la respuesta
print(response.status_code)
print(response.text)

200
<p>Message to: +34653508040<p>Text to send: 🤖¡Ejecución terminada! 🤖%0AVe a echarle un vistazo ☝🤓<p><b>Message queued.</b> You will receive it in a few seconds.


In [15]:
!zip -r results_temas_bert.zip results_temas_bert
from IPython.display import FileLink 
FileLink(r'results_temas_bert.zip')

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  adding: results_temas_roberta/ (stored 0%)
  adding: results_temas_roberta/res_temas_roberta_0.txt (deflated 57%)
  adding: results_temas_roberta/res_temas_roberta_1.txt (deflated 57%)
  adding: results_temas_roberta/res_temas_roberta_4.txt (deflated 55%)
  adding: results_temas_roberta/res_temas_roberta_3.txt (deflated 57%)
  adding: results_temas_roberta/res_temas_roberta_5.txt (deflated 60%)
  adding: results_temas_roberta/res_temas_roberta_2.txt (deflated 60%)


/kaggle/working/results_temas_roberta.zip